In [1]:
from matplotlib import pyplot as plt
from fed_algo import FedAvg, FedAdam, FedAdagrad, FedYogi, FedAlgo, ClientParam, Layers
import pandas as pd
import numpy as np
from typing import Tuple, List
from utils import read_chunk, select_features_name, read_raw_dataset, train_with_data
from model import gru

2023-01-03 22:53:03.552724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 22:53:03.724555: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-03 22:53:04.468422: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-03 22:53:04.468587: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# pokemon


In [2]:
X_raw, y = read_raw_dataset("datasets/pokemon.csv",
                            index_col='id',
                            target_name='MOS',
                            drop=['user_id'])
features = select_features_name(X_raw, y)
X_test, y_test = read_chunk('pokemon', '-test', features)
X_init, y_init = read_chunk('pokemon', '-init', features)
result = {
    'num_clients': [25, 50, 75, 100, 125, 150],
    'central': [],
    'FedAvg': []
}

init models


In [3]:
central_model = gru()
central_model.fit(X_init,
                  y_init,
                  validation_split=0.05,
                  batch_size=128,
                  epochs=20,
                  shuffle=True)
                  
init_weights = central_model.get_weights()

fed_avg = FedAvg(gru(), init_weights, X_test, y_test)


2023-01-03 22:53:05.604779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:53:05.622586: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:53:05.622997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-03 22:53:05.623943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them i

Epoch 1/20


/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]
2023-01-03 22:53:09.409805: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-01-03 22:53:09.536844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-03 22:53:09.540661: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fb4700b5370 initialized for

1/1 [==============================] - 4s 4s/step - loss: 10.2481 - mean_squared_error: 10.2481 - val_loss: 6.1613 - val_mean_squared_error: 6.1613
Epoch 2/20
1/1 [==============================] - 0s 36ms/step - loss: 4.3445 - mean_squared_error: 4.3445 - val_loss: 2.1645 - val_mean_squared_error: 2.1645
Epoch 3/20
1/1 [==============================] - 0s 36ms/step - loss: 2.0761 - mean_squared_error: 2.0761 - val_loss: 0.4770 - val_mean_squared_error: 0.4770
Epoch 4/20
1/1 [==============================] - 0s 34ms/step - loss: 1.5711 - mean_squared_error: 1.5711 - val_loss: 0.7036 - val_mean_squared_error: 0.7036
Epoch 5/20
1/1 [==============================] - 0s 35ms/step - loss: 2.3769 - mean_squared_error: 2.3769 - val_loss: 1.1068 - val_mean_squared_error: 1.1068
Epoch 6/20
1/1 [==============================] - 0s 33ms/step - loss: 2.9964 - mean_squared_error: 2.9964 - val_loss: 0.9084 - val_mean_squared_error: 0.9084
Epoch 7/20
1/1 [==============================] - 0s 32ms

In [4]:

X_central = pd.DataFrame()
y_central = pd.Series()

/tmp/ipykernel_8817/3602875039.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_central = pd.Series()


In [5]:
num_clients = [25, 50, 75, 100, 125, 150]
for num_client in num_clients:

    datasets = [
        read_chunk('pokemon',
                   chunk_id + 1 + num_client - 25, features)
        for chunk_id in range(25)
    ]
    X_central = pd.concat([X_central] + [X for X, _ in datasets], axis=0)
    y_central = pd.concat([y_central] + [y for _, y in datasets], axis=0)

    # centralized training
    central_model.set_weights(init_weights)
    central_model.fit(X_central,
                      y_central,
                      validation_split=0.05,
                      batch_size=128,
                      epochs=20,
                      shuffle=True)

    # federated training

    local_models = [
        train_with_data(gru(), fed_avg.get_weights(), dataset)
        for dataset in datasets
    ]
    fed_avg.aggregate(local_models)

    central_mse = central_model.evaluate(X_test, y_test)[0]
    fed_mse = fed_avg.predict()

    result['central'].append(central_mse)
    result[fed_avg.name()].append(fed_mse)

Epoch 1/20
1/2 [==============>...............] - ETA: 0s - loss: 1.0950 - mean_squared_error: 1.0950

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


2/2 [==============================] - 0s 61ms/step - loss: 0.9476 - mean_squared_error: 0.9476 - val_loss: 0.5093 - val_mean_squared_error: 0.5093
Epoch 2/20
2/2 [==============================] - 0s 30ms/step - loss: 1.1083 - mean_squared_error: 1.1083 - val_loss: 0.6120 - val_mean_squared_error: 0.6120
Epoch 3/20
2/2 [==============================] - 0s 38ms/step - loss: 1.0060 - mean_squared_error: 1.0060 - val_loss: 0.5408 - val_mean_squared_error: 0.5408
Epoch 4/20
2/2 [==============================] - 0s 38ms/step - loss: 1.0606 - mean_squared_error: 1.0606 - val_loss: 0.4461 - val_mean_squared_error: 0.4461
Epoch 5/20
2/2 [==============================] - 0s 43ms/step - loss: 0.7845 - mean_squared_error: 0.7845 - val_loss: 0.4219 - val_mean_squared_error: 0.4219
Epoch 6/20
2/2 [==============================] - 0s 39ms/step - loss: 1.0128 - mean_squared_error: 1.0128 - val_loss: 0.4517 - val_mean_squared_error: 0.4517
Epoch 7/20
2/2 [==============================] - 0s 37ms

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 3s 3s/step - loss: 0.8148 - mean_squared_error: 0.8148 - val_loss: 0.1574 - val_mean_squared_error: 0.1574
Epoch 2/20
1/1 [==============================] - 0s 45ms/step - loss: 0.6633 - mean_squared_error: 0.6633 - val_loss: 0.2992 - val_mean_squared_error: 0.2992
Epoch 3/20
1/1 [==============================] - 0s 42ms/step - loss: 0.8166 - mean_squared_error: 0.8166 - val_loss: 0.2030 - val_mean_squared_error: 0.2030
Epoch 4/20
1/1 [==============================] - 0s 43ms/step - loss: 0.8099 - mean_squared_error: 0.8099 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 5/20
1/1 [==============================] - 0s 42ms/step - loss: 0.4243 - mean_squared_error: 0.4243 - val_loss: 0.2540 - val_mean_squared_error: 0.2540
Epoch 6/20
1/1 [==============================] - 0s 46ms/step - loss: 0.7098 - mean_squared_error: 0.7098 - val_loss: 0.6462 - val_mean_squared_error: 0.6462
Epoch 7/20
1/1 [==============================] - 0s 44ms/s

...

1/1 [==============================] - 4s 4s/step - loss: 0.8287 - mean_squared_error: 0.8287 - val_loss: 0.1234 - val_mean_squared_error: 0.1234
Epoch 2/20
1/1 [==============================] - 0s 52ms/step - loss: 2.0267 - mean_squared_error: 2.0267 - val_loss: 0.8746 - val_mean_squared_error: 0.8746
Epoch 3/20
1/1 [==============================] - 0s 59ms/step - loss: 0.2187 - mean_squared_error: 0.2187 - val_loss: 1.6818 - val_mean_squared_error: 1.6818
Epoch 4/20
1/1 [==============================] - 0s 54ms/step - loss: 1.5013 - mean_squared_error: 1.5013 - val_loss: 2.1534 - val_mean_squared_error: 2.1534
Epoch 5/20
1/1 [==============================] - 0s 68ms/step - loss: 1.3646 - mean_squared_error: 1.3646 - val_loss: 1.9756 - val_mean_squared_error: 1.9756
Epoch 6/20
1/1 [==============================] - 0s 50ms/step - loss: 1.2256 - mean_squared_error: 1.2256 - val_loss: 1.4020 - val_mean_squared_error: 1.4020
Epoch 7/20
1/1 [==============================] - 0s 44ms/s

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 3s 3s/step - loss: 1.4876 - mean_squared_error: 1.4876 - val_loss: 0.4838 - val_mean_squared_error: 0.4838
Epoch 2/20
1/1 [==============================] - 0s 59ms/step - loss: 0.5867 - mean_squared_error: 0.5867 - val_loss: 0.9764 - val_mean_squared_error: 0.9764
Epoch 3/20
1/1 [==============================] - 0s 59ms/step - loss: 0.7849 - mean_squared_error: 0.7849 - val_loss: 0.5629 - val_mean_squared_error: 0.5629
Epoch 4/20
1/1 [==============================] - 0s 68ms/step - loss: 0.3054 - mean_squared_error: 0.3054 - val_loss: 0.2681 - val_mean_squared_error: 0.2681
Epoch 5/20
1/1 [==============================] - 0s 55ms/step - loss: 0.6096 - mean_squared_error: 0.6096 - val_loss: 0.1020 - val_mean_squared_error: 0.1020
Epoch 6/20
1/1 [==============================] - 0s 48ms/step - loss: 1.6215 - mean_squared_error: 1.6215 - val_loss: 0.0361 - val_mean_squared_error: 0.0361
Epoch 7/20
1/1 [==============================] - 0s 53ms/s

/home/ducnm/.local/lib/python3.8/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1/1 [==============================] - 2s 2s/step - loss: 0.5963 - mean_squared_error: 0.5963 - val_loss: 0.0070 - val_mean_squared_error: 0.0070
Epoch 2/20
1/1 [==============================] - 0s 44ms/step - loss: 0.4755 - mean_squared_error: 0.4755 - val_loss: 0.0617 - val_mean_squared_error: 0.0617
Epoch 3/20
1/1 [==============================] - 0s 41ms/step - loss: 0.6051 - mean_squared_error: 0.6051 - val_loss: 0.3959 - val_mean_squared_error: 0.3959
Epoch 4/20
1/1 [==============================] - 0s 43ms/step - loss: 0.4467 - mean_squared_error: 0.4467 - val_loss: 0.9220 - val_mean_squared_error: 0.9220
Epoch 5/20
1/1 [==============================] - 0s 43ms/step - loss: 0.7854 - mean_squared_error: 0.7854 - val_loss: 0.7960 - val_mean_squared_error: 0.7960
Epoch 6/20
1/1 [==============================] - 0s 43ms/step - loss: 0.2292 - mean_squared_error: 0.2292 - val_loss: 0.5322 - val_mean_squared_error: 0.5322
Epoch 7/20
1/1 [==============================] - 0s 45ms/s

In [6]:
import json

json_result = json.dumps(result, indent=4)
print(json_result)

{
    "num_clients": [
        25,
        50,
        75,
        100,
        125,
        150
    ],
    "central": [
        0.27947115898132324,
        0.23901864886283875,
        0.24239014089107513,
        0.2248125672340393,
        0.25256383419036865,
        0.2530658543109894
    ],
    "FedAvg": [
        0.367812842130661,
        0.3604499101638794,
        0.313906729221344,
        0.28327444195747375,
        0.28571224212646484,
        0.26367688179016113
    ]
}
